In [29]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import interpolate
import math
from scipy import signal
from numba import jit
%run StreamClass.ipynb

## Plot each variable on a graph

## Kalman Filter Implementation

In [3]:
%load_ext line_profiler

def getQ(Q, acc_stddev, dt):
    vel_stddev = acc_stddev*dt
    dis_stddev = 0.5*acc_stddev*dt**2
    Q[0, 0] = dis_stddev**2
    Q[1, 1] = dis_stddev**2
    Q[2, 2] = vel_stddev**2
    Q[3, 3] = vel_stddev**2
    Q[0, 2] = dis_stddev*vel_stddev
    Q[1, 3] = dis_stddev*vel_stddev
    return Q


def getFkBk(Fk, Bk, dt):
    Fk[0, 2] = dt
    Fk[1, 3] = dt
    Bk[0, 0] = 0.5*dt**2
    Bk[1, 1] = 0.5*dt**2
    Bk[2, 0] = dt
    Bk[3, 1] = dt
    return Fk, Bk

def kalman_predict(dt, Fk, Bk, xk, uk, Pk, Q, acc_stddev):
    Q = getQ(Q, acc_stddev, dt)
    Fk, Bk = getFkBk(Fk, Bk, dt)
    xk = np.matmul(Fk, xk) + np.matmul(Bk, uk)
    Pk = np.matmul(np.matmul(Fk, Pk), Fk.T) + Q
    return xk, Pk, Fk, Bk

def kalman_gain(Pk, H, R):
    PkHT = np.matmul(Pk, H.T)
    K = np.matmul(PkHT, np.linalg.inv(np.matmul(H, PkHT)+R))
    return K

def kalman_update_uncertainty(Pk, K, H):
    return Pk - np.matmul(K, np.matmul(H, Pk))

def kalman_update_state(xk, zk, K, H):
    Hxk = np.matmul(H, xk)
    return xk+np.matmul(K, zk-Hxk )



def reverse_data(acc, gps):
    
    end_time = acc[-1, 0]
    acc = acc[::-1]
    gps = gps[::-1]
    
    for i in range(len(acc)):
        acc[i] = -1*acc[i]
        acc[i, 0] += end_time
        gps[i, 0] = acc[i, 0]
        
    return acc.T, gps.T

def compute_kalman(data, reverse=False):
    ## Assume that the initial state is position (0, 0) and velocity (0, 0)       
    acc = data.acc_ERC.T
    gps = data.gps.T
    
    if(reverse):
        acc, gps = reverse_data(acc, gps)
    gps = np.concatenate((gps[1:3, :], np.zeros((2, gps.shape[1]))))
    
    
    x0 = np.asmatrix([gps[0, 0], gps[1, 0], 0, 0]).T
    xks = [x0]
    u0 = acc[1:3, 0]
    uks = [u0]
    Fk = np.asmatrix(np.identity(4))
    Bk = np.zeros((4, 2))

    acc_stddev = 0.6

    P0 = np.zeros((4, 4))
    Pks = [P0]

    H = np.identity(4)
    H[2, 2] = 0.0
    H[3, 3] = 0.0
    gps_var = 15.2
    R = gps_var*np.identity(4)
    Q = np.zeros((4,4))
    for i in range(1, acc.shape[1]):
        dt = acc[0, i] - acc[0, i-1]
        xk, Pk, Fk, Bk = kalman_predict(dt, Fk, Bk, xks[-1], acc[1:3, i-1], Pks[-1], Q, acc_stddev)
        xks.append(xk)
        Pks.append(Pk)

        K = kalman_gain(Pks[-1], H, R)
        xks[-1] = kalman_update_state(xk, gps[:, i], K, H)
        Pks[-1] = kalman_update_uncertainty(Pks[-1], K, H)
        

    xks = np.concatenate(xks, axis=1).T
    data.init_kalman(xks, reverse)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
